# Assembly Lab

- Authors:
  - Daniel Akselrod, akselrod@mcmaster.ca
  - Jonathan Avraham, avrahamj@mcmaster.ca
- Group ID on Avenue: Assembly 51
- Gitlab URL: https://gitlab.cas.mcmaster.ca/akselrod/l3-assembly

### F1: Global Variables and First Visits

All Manual translations, including the ones done for F2, F3, and F4, will be completed in the Pep9-Files folder. 

##### Manual Translation

##### Global and Local Variables

A global variable is a variable that is declared outside of any function in a program, and can be accessed and used by any function in the program. This means that any changes made to a global variable will affect all functions that use that variable. On the contrary, a local variable is a variable that is declared inside of a function, and can only be accessed and used by that function. 

Given an RBS program, one can determine whether it is a global or local variable by looking at the scope in which it was declared. If the variable is declared outside of any function, it is a global variable. However, if it is declared inside of a function, it is a local variable.

##### NOP1 Instructions

NOP1 instructions are used in the translator, the keyword, NOP1, indicating "No Operation" in PEP/9, is used in the translator due to the requirements of the method __record_instruction(). This method takes in two arguements, an instruction and its associated label. Given a certain scenerio, this method would need to be invoked when a label is created but no operation is wanted. Inserting an NOP1 instruction would allow for no opertion to occour when this instruction is called, serving as a sort of placeholder for the code. The NOP1 instruction can also be used to delay operations.

##### Visitors and Generators

The translator.py relies on two visitors being the GlobalVariableExtraction and the TopLevelProgram, in addition to two generators being the EntryPoint and the StaticMemoryAllocation. The purpose of the two visitors is to visit the nodes in the ast in order to actually generate the assembly code. The purpose of the two generators is to take the information gathered by the visitors and then actually generate/print the output of the assembly code. To dive deeper into the visitors, we first have the global variable extraction. This visitor class is used to extract the global variables from the ast tree by visiting the Assign node. When one is encountered, it extracts the id from the node, which is the variable name, and adds it to a set of global variables. For the second visitor, the TopLevelProgram visits all the nodes from the ast and it will record certain instructions to a list that can later be generated. These instrcutions of composed of the variable names, the instructions themselves, and labels. The StaticMemoryAllocation generator generates the global memory allocation for global variables that is passed in by the visitor. These global variables then get generated by printing the variable name, with its memory allocation beside it (.BLOCK 2). The Entry level generator generates the top level instructions by taking in the recording instructions by the visitor and then prints out the instructions. 

#### Limitations

As of now, the current translation code has many limitations which disables it from translating certain aspects of RBS python code into Pep/9. First of all, the current translator only accounts for global variables and allocates memory in an inefficient way. In addition to this, the current translator does not translate certain parts of the RBSlanguage like conditional statements and array objects. This is because the visitors does not account for visiting conditionals in AST and lists. We would need to create more methods and classes to visit the nodes, and create more visitors to account for not only top level programs, but lower level programs like function calling. 

### F2: Allocation, Constants, and Symbols

#### Explanation of Improvement

##### Memory Allocation and Constants
For the improvement of memory allocation, a change was made in the visit GlobalVariables and the generator StaticMemoryAllocation. For the global variable extraction, the self.results variable is no longer initialized with a set, and is now initialized by a dictionary. This way, we can map each global variable with its known integer value, and can use it when generating the global variable. The global variable extractor now extracts the global variables by visiting the Assign node, and extracts the value (if its known) by visiting the visit_Constant method which was added. If the constant value is not existent, then the global variable will map to None in the self.results dictionary. Finally, this is generated in the StaticMemoryAllocation by looping through each key in the dictionary and printing .WORD beside the variable with its value if its mapped to value, and if the variable is mapped to None, then we print .Block 2 like before. In the top level program class, we will omit the redundant LDWA and STWA which is done in the visit_Assign method by skipping a section of the code if the variable has a known constant. In addition to improving the memory allocation, we also improved the translator by accounting for constants, which does not change its value once it is set if the id has a leading underscore and uppercase letters. The only fix to this was in the StaticMemoryAllocation generator, where we check if the global variable (key in dict) has these properties. If it does, then instead of writing .WORD with its corresponding value, we write .EQUATE with its value. This is also changed in the TopLevelProgram where we use the __access_memory() method. If the id has a leading underscore and all uppercase, then we add the instuction with the node.id, stating that its an immediate value rather than direct from memory.

##### Symbol Table
The symbol table is used in the Python RBS to PEP/9 translator to condense variable names to ensure that they are less than 8 characters in length, the maximum length of a variable in PEP/9. In order to achieve this, the intial plan was to strip variable names of their vowels and then trim it to a length of 8 if still longer than 8 characters. However, upon realizing that this would not work, variable names with only vowels were only trimmed to their original size. This is not an ideal solution as two variables such as "BCDFGHJKA" and "BCDFGHJKB" would both be left the same name. A more ideal solution would be to keep track of all added variables and change the name if any modified name has already occoured. For example, in this scenerio the last index can be incremented by 1.

#### Overflow Handling

Overflows can be an issue when dealing with very large variable values. Many times, overflows have to manually checked for and resolved. For example, every itteration of a fibbonaci recursive call may first ensure that the sum of the two addings terms are less than the data size of the resprctive type. This will be true for low input values of the fibbonaci function but can quickly exceed the maximum storage size. 

Additionally, many programming languages, including Python Java, and C++, suport the BigInt type which essentially is an integer value that can grow in memory size depending on the value in which it holds. This type ensures that programmers do not have to worry about any potential overflow issue and how it is handeled.

### F3: Conditionals

##### Automating the translation of Conditional statements

The TopLevelProgram class will automate the translation of conditionals. When translating the gcd.py file into its AST, we can see that conditionals are visited using the ast.If node visitor. Inside this node, there is a section for testing the comparison, a section for the body, and a section for the orelse block. Therefore, in the TopLevelProgram class, we will implement a visit_If method to handle conditionals. This method will be similar to the visit_While method, as it uses branches and comparisons. First, we will add a conditional ID to be used for branching. Then, we will create a map called inverted that maps the boolean operator in the ast to its inverted operator for branching. When visiting the conditional, we will load a value and compare it, then branch appropriately. If the orelse section is empty, we will branch to the end_if, otherwise we will branch to the else_ID. Then, we will visit the contents of the body and branch to the end_if if a section of the conditional was executed. Finally, we will visit the contents of the orelse and finish the method by adding the end_if label at the end. This change only impacts the Entry Point generator, as it reads all the instructions in the ToplevelProgram, which now accounts for visiting conditionals.

### F4: Function Calls

##### Ranking Programs by Translation Complexity

In terms of translation complexity from lowest to highest, the rank is as follows: call_void, call_param, call_return, fibonnaci, fib_rec, factorial, factorial_dec. The reason to why call_void is lowest on the list is because it does even need to add stack pointers in the main program, only in the function itself as it has local variables. It does not have a return value or parameters, meaning that it does not need to initialize a stack pointer in main. Call_param is next as it has to call parameters meaning that the parameter needs to be initialized in the stack pointer, before the return address. Next, the call_return is more complex than the call_param as it also has parameters, but also has a return value which needs to be included in the stack in the main program. Next we have fibannoci, which is a bit more complex in translating compared to the last one since it contains the while loop which is part of the TopLevelProgram. Next we have fib_rec, which is more complex as it has functions inside functions (it is recursive). This means that there is a lot more that needs to be added to the stack. Similarily, we have the factorial.py which has while loops and functions called inside other functions. Finally, we have factorial_rec which is like factorial.py, but has the addition of recursion. 

##### Automating the translation of Function Calls

To transalte the function definitions in RBS, we would need to visit the ast.FunctionDef. However, we do not want to visit this node in the TopLevelProgram class as function calling is not considered a top level program. Instead, we need to introduce a new visitor, potentially called FunctionCalling, which is a new visitor class that inherits the ast.NodeVistor parent class and visits the ast accordingly. This act very similar to the TopLevelProgram class, however will have the visit_FunctionDef method inside it. This visitor class will be instatiated in the transalor.py by adding it right after calling the StaticMemoryAllocation, and right before the TopLevelProgram as it is lower level and should be called first. In this class, we have a self.instructions list which will record the instructions that is in the functions, and will be later generated in a new generator class that generates the functions in the program. Another part of the program that will need to be changed is the global variable extraction. Instead, we now want to extract local variables from the functions and add it to a set of local_variables where it can be generated in the StaticMemoryAllocation. Here, the generator would add .EQUATE n stack allocations to the local variables and print it out in the generator.  

##### Stack Overflow

A stack overflow happens when a program tries to store more data on the stack than the stack's capacity. This can happen if the program has an infinite loop that keeps pushing data onto the stack, or if it makes a lot of recursive function calls without properly cleaning up the stack. When a stack overflow occurs, the program may crash or halt, and the operating system may show an error message. In some cases, the stack overflow can cause the program to behave in unpredictable ways. To prevent stack overflow, it's important to manage the stack properly by limiting the amount of data on the stack and properly cleaning up the stack after each function call. In some cases, it may be necessary to use a larger stack or implement a mechanism to prevent the stack from becoming full.

### F5: Arrays

To account for arrays globally, we need to need to modify the GlobalVariableExtraction. To do this, we need to add a visit_Binops as there is a multiplication when we instatiate an array, and extract that constant. This constant determines the length of the array and how large the index register needs to be. Then, when we extract the global variables, we need to pair the variable name with '.Block n' where 'n' is the constant multiplied by 2 since one index in the array is an integer and represents two memory slots. Therefore, we would also need to change the StaticMemoryAllocation class to generate and print the global array variable with '.Block n'. In addition to this, we need to change the top level program as we make changes to the array. Here, we would add a new method called visit_Subscript, which is where we add the LDWX, CPWX, and STWX instructions and where the index register is actually being used. 

### Self Reflection

#### Daniel Akselrod

##### Backwards
Before the assignment, I had a relatively good understanding on the underlying fundamentals of computer architecture due to our earnings last semester. As a result, I found certain aspects of the easier such as understanding how Assembly code worked although PEP/9 was a new language. Additionally, I did not know about the AST module and how its representation tree can be used to translate between programming languages given correct instructions. 
##### Inwards
Things that I found frustrating about the assignment would definitely have to include navigating the AST tree. Understanding its structure and how it stored information such as assignments, operations, and functions definitely took a lot of time and understanding. Additionally, the general task of working in an assembly language is very tedious as it is very low level and takes a lot of understanding.
##### Outwards
As an instructor, I believe that this Lab was very interesting and would be good preparation for students leading up to the exam. Additionally, it encourages us to use resources and think very hard which is always what one wants in a lab. I would also encourage students to make use of provided materials such as lecture videos, notes, and online content.
##### Forwards
If I had the chance to restart the assignment, I would definitely give myself more time as I underestimated the complexity of F4 and F5. Additionally, I would dedicate more time to understanding how the AST module works with a very large understanding of the tree as that would have very much improved my capability to analyze the tree when it came to jumping branches for conditionals and functions.

#### Jonathan Avraham
##### Backwards
Before we started, I had a basic understanding of the subject. I didn't know about RBS python and the AST module but was familar with other assembly code we've looked at in previous years. Additionally, I was not familiar with the process of translating code between languages but I took this as an opportunity to learn a lot and have some fun, though this fun would quickly change.
##### Inwards
One thing that I found frustrating about the assignment was working with the AST tree. It was difficult to understand its structure and how it stored information, and it took a lot of time and effort to figure it out. This was espeically true when it came to function calling as branching between different code segments was personally really hard. I also found working in an assembly language to be challenging because it is very low-level and requires a lot of attention to detail, especially given the fact that this was a new language for me as last year we worked in ARM LEGv*.
##### Outwards
If I were the instructor, I would say that this lab was a great learning experience and a good way to apply the concepts we learned in class to a real-world scenario. I would also encourage students to use the available resources and to ask for help if they need it.Perhaps more academic content on the groundwork of the AST module could have been provided and that would have left students with a greater understanding of the relationships between python and its AST representation.
##### Forwards
If I had a chance to do the assignment again, I would give myself more time to work on it. I would also spend more time learning about the AST module and getting a better understanding of how it works. This would have made it easier for me to navigate the tree structure and complete the tasks. Overall, I think that the lab was a valuable learning experience, but it was also challenging and required a lot of effort.